<a href="https://colab.research.google.com/github/mirianfsilva/machine-learning/blob/master/deep-learning/transferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning
### Practical II - Machine Learning

- Objectives: 
The goal of this practical is to practice the use of Convolutional Neural Networks (CNNs) and Transfer Learning. In this practical you will compare three strategies for training CNNs, two of which use Transfer Learning, in an image classification problem. 

- Task:
Train a CNN to classify the images from part A using the Keras API for TensorFlow. 

- Dataset:  CIFAR-101 . This dataset has 60000 images of 32x32 pixels, of which 50000 are for training and 10000 for test. Each image belongs to one of 10 classes, numbered from 0 to 9. The classes are: airplane, automobile, bird, cat, deer, dog, frog,
horse, ship and truck. 

- References: [The CIFAR-10 dataset](https://www.cs.toronto.edu/~kriz/cifar.html)


In [0]:
import matplotlib as mpl
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from IPython.display import display
from IPython.display import Markdown

from sklearn import metrics
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

from keras import datasets
from keras import layers
from keras import optimizers
from keras import models

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

def printmd(s):
    display(Markdown(s))

In [0]:
from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()